In [1]:
!rm -rf self-expanding-nets
!git clone https://github.com/CTLab-ITMO/self-expanding-nets
%pip uninstall senmodel
%pip install -U -e ./self-expanding-nets/

Cloning into 'self-expanding-nets'...
remote: Enumerating objects: 1061, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 1061 (delta 118), reused 117 (delta 54), pack-reused 868 (from 1)
Receiving objects: 100% (1061/1061), 2.41 MiB | 5.00 MiB/s, done.
Resolving deltas: 100% (638/638), done.
Obtaining file:///content/self-expanding-nets
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8

## Imports

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, TensorDataset
from torchvision import datasets, transforms
import os
import random
import numpy as np

SEED = 0
torch.manual_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
g = torch.Generator()
g.manual_seed(0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'  # or ':16:8'


################################
#     RESTART     RUNTIME      #
################################
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

## Data

In [3]:
BATCH_SIZE = 128

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

train_dataset = datasets.FashionMNIST(root='./data', train=True,
                                      download=True, transform=transform)
val_dataset = datasets.FashionMNIST(root='./data', train=False,
                                    download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.7MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 201kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.72MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 9.89MB/s]


## Model

In [4]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=28 * 28, hidden_size=16, output_size=10):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = self.fc0(x)
        return x

In [5]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0], device=device)

## Train

In [6]:
hyperparams = {
    'num_epochs': 64,
    'metric': AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    'aggregation_mode': 'mean',
    'choose_thresholds': {'fc0': 0.6},
    'replace_layers': ['fc0'],
    'threshold': 0.05,
    'min_delta_epoch_replace': 8,
    'window_size': 5,
    'lr': 0.00012,
    'delete_after': 2,
    'task_type': 'classification',
    'max_to_replace': None,
    'choose_thresholds_del': {'fc0': 0.02},
    'fully_connected': False
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 64, metric: AbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc0': 0.6}, replace_layers: ['fc0'], threshold: 0.05, min_delta_epoch_replace: 8, window_size: 5, lr: 0.00012, delete_after: 2, task_type: classification, max_to_replace: None, choose_thresholds_del: {'fc0': 0.02}, fully_connected: False"

In [7]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: down-shift to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
wandb.finish()
run = wandb.init(
    project="self-expanding-nets",
    name=f"{name}",
)

In [ ]:
criterion = nn.CrossEntropyLoss()
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, criterion, hyperparams, device)

100%|██████████| 469/469 [00:08<00:00, 52.33it/s]


Epoch 1/64, Train Loss: 1.3163, Val Loss: 0.9454, Val Accuracy: 0.7051


100%|██████████| 469/469 [00:08<00:00, 58.03it/s]


Epoch 2/64, Train Loss: 0.8291, Val Loss: 0.7658, Val Accuracy: 0.7487


100%|██████████| 469/469 [00:08<00:00, 56.57it/s]


...

100%|██████████| 469/469 [00:10<00:00, 44.36it/s]


Epoch 18/64, Train Loss: 0.3754, Val Loss: 0.4055, Val Accuracy: 0.8552


100%|██████████| 469/469 [00:10<00:00, 44.55it/s]


Epoch 19/64, Train Loss: 0.3650, Val Loss: 0.4005, Val Accuracy: 0.8583
Chosen edges: tensor([[   4,    6,    2,    0,    4,    2,    4,    6,    2],
        [ 786,  786, 1607, 1623, 1623, 2017, 2017, 2017, 2068]],
       device='cuda:0') 9


 10%|▉         | 45/469 [00:01<00:09, 44.37it/s]